# Pansharpening GANs -- training set creation

## Setup

In [8]:
import os
import pandas as pd
import numpy as np

### Process raw rasters

In [14]:
# read all files
inp_folder = "raw_scenes"
raster_files = [f"{inp_folder}/{scn}" for scn in os.listdir(inp_folder) if scn.split('.')[-1] == 'tif']


# separate by panchromatic and multispectral
raster_pan = [scn for scn in raster_files if 'P1BS' in scn]
raster_ms = [scn for scn in raster_files if 'M1BS' in scn]


# process scenes
for scn_pan in raster_pan:
    scn_ms = f"{scn_pan.split('-')[0]}-M1BS-{scn_pan.split('-')[2]}"
    # check if matching multispectral scene is present
    if scn_ms in raster_ms:
        !python generate_training_scene.py --pan_scene=$scn_pan \
                                           --ms_scene=$scn_ms \
                                           --out_folder="processed_scenes"

### Tile processed rasters

In [17]:
# find all panchromatic processed rasters
processed_pan = [scn for scn in os.listdir('processed_scenes') if 'pan' in scn]

# tile raster pairs (split into training and validation)
prob_val = 0.1
for scn_pan in processed_pan:
    if np.random.random() > prob_val:
        !python tile_training_scene.py --file_pan=$scn_pan --dataset="training" \
                                       --patch_size=336 --stride=0.7
    else:
        !python tile_training_scene.py --file_pan=$scn_pan --dataset="validation" \
                                       --patch_size=224 --stride=1